Fase 1: EXTRACCION DE DATOS DE PELICULAS
Nuestro primer objetivo es extraer los datos de películas de la API. En Adalab nos hemos creado un API
muy sencilla que contiene la siguiente información:
Datos a extraer:
Título
Año de lanzamiento
Duración (en minutos)
Género
Contenido para adultos (sí o no)
El objetivo es extraer 100 películas de esta API utilizando el siguiente endpoint:
https://beta.adalab.es/resources/apis/pelis/pelis.json

In [ ]:
## Importar las librerias necesarias
import requests ## para conectar con Api y descargar los datos
import pandas as pd ## para convertir los datos en formato tabla
import mysql.connector ## para conectar Paython con MySQL
import numpy as np ## para convertir formato nan de Paython a none de MySQL
from mysql.connector import Error ## captura errores de MySQL

In [9]:
## Función para llamar a Api y comprobar errores

def llamar_api(url):
    datos = requests.get(url)       ## llamamos a la APi
    print(f"Llamada al API, con respuesta : {datos.status_code}") ## muestra si la respuesta es correcta

    if datos.status_code != 200:
        print(f"Llamada falló, por el siguiente motivo: {datos.reason}")## si hay error nos indica el motivo
    else:
        return datos.json() ## nos devuelve los datos en formato json

In [10]:
## Extraer datos de la Api

url = "https://beta.adalab.es/resources/apis/pelis/pelis.json"
datos_pelis = llamar_api(url)

Llamada al API, con respuesta : 200


In [12]:
## Muestra de los datos extraidos
datos_pelis
type(datos_pelis)

list

In [13]:
## Datos_pelis es una lista de diccionarios;elimino la clave:valor id
for peli in datos_pelis:
    if "id" in peli:
       del peli ["id"]
print(datos_pelis)

[{'titulo': 'The Godfather', 'año': 1972, 'duracion': 175, 'genero': 'Crimen', 'adultos': False, 'subtitulos': ['es', 'en']}, {'titulo': 'The Godfather Part II', 'año': 1974, 'duracion': 202, 'genero': 'Crimen', 'adultos': False, 'subtitulos': ['es', 'en']}, {'titulo': 'Pulp Fiction', 'año': 1994, 'duracion': 154, 'genero': 'Crimen', 'adultos': True, 'subtitulos': ['es', 'en']}, {'titulo': 'Forrest Gump', 'año': 1994, 'duracion': 142, 'genero': 'Drama', 'adultos': False, 'subtitulos': ['es', 'en', 'fr']}, {'titulo': 'The Dark Knight', 'año': 2008, 'duracion': 152, 'genero': 'Acción', 'adultos': False, 'subtitulos': ['es', 'en']}, {'titulo': 'Fight Club', 'año': 1999, 'duracion': 139, 'genero': 'Drama', 'adultos': True, 'subtitulos': ['es', 'en']}, {'titulo': 'Inception', 'año': 2010, 'duracion': 148, 'genero': 'Ciencia ficción', 'adultos': False, 'subtitulos': ['es', 'en', 'de']}, {'titulo': 'The Matrix', 'año': 1999, 'duracion': 136, 'genero': 'Ciencia ficción', 'adultos': False, 'sub

In [20]:
## Convierto datos_pelis a tabla a través de pandas y muestro los resultados
tabla_datos_pelis = pd.DataFrame(datos_pelis)
tabla_datos_pelis

,titulo,año,duracion,genero,adultos,subtitulos
0,The Godfather,1972,175,Crimen,False,"[es, en]"
1,The Godfather Part II,1974,202,Crimen,False,"[es, en]"
2,Pulp Fiction,1994,154,Crimen,True,"[es, en]"
3,Forrest Gump,1994,142,Drama,False,"[es, en, fr]"
4,The Dark Knight,2008,152,Acción,False,"[es, en]"
...,...,...,...,...,...,...
95,La vita è bella,1997,116,Drama,False,"[es, en, it]"
96,Requiem for a Dream,2000,102,Drama,True,"[es, en]"
97,Memento,2000,113,Thriller,True,"[es, en]"
98,Eternal Sunshine of the Spotless Mind,2004,108,Drama,False,"[es, en]"


FASE 2: CREACIÓN DE LA BASE DE DATOS


In [36]:
## Conexión de Paython a MySQL
## Conexión entre Paython y MySQL
try:
    cnx = mysql.connector.connect(
        host='127.0.0.1',
        user='root',
        password='111729',
    )
    print('Conexión exitosa')
except Error as tipo_error:
    print('Error al conectar:', tipo_error)

Conexión exitosa


In [37]:
## Crear base de datos desde Paython
try:
    mycursor = cnx.cursor()
    query = "CREATE DATABASE IF NOT EXISTS bd_peliculas_adalab "
    mycursor.execute(query)
    print("Query exitosa")
except :
    print("Error.")

Query exitosa


In [38]:
## Antes de crear la tabla en la base de datos compruebo como son los datos que tengo
tabla_datos_pelis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   titulo      100 non-null    object
 1   año         100 non-null    int64 
 2   duracion    100 non-null    int64 
 3   genero      100 non-null    object
 4   adultos     100 non-null    bool  
 5   subtitulos  100 non-null    object
dtypes: bool(1), int64(2), object(3)
memory usage: 4.1+ KB


In [39]:
## Compruebo si tiene algun null, si es así se cambia el formato para insertar en MySQL
tabla_datos_pelis.isnull().sum()

titulo        0
año           0
duracion      0
genero        0
adultos       0
subtitulos    0
dtype: int64

In [40]:
## Cambio el nombre de una columna de la tabla
tabla_datos_pelis_act = tabla_datos_pelis.rename(columns={"adultos": "contenido_para_adultos"})
tabla_datos_pelis_act

,titulo,año,duracion,genero,contenido_para_adultos,subtitulos
0,The Godfather,1972,175,Crimen,False,"[es, en]"
1,The Godfather Part II,1974,202,Crimen,False,"[es, en]"
2,Pulp Fiction,1994,154,Crimen,True,"[es, en]"
3,Forrest Gump,1994,142,Drama,False,"[es, en, fr]"
4,The Dark Knight,2008,152,Acción,False,"[es, en]"
...,...,...,...,...,...,...
95,La vita è bella,1997,116,Drama,False,"[es, en, it]"
96,Requiem for a Dream,2000,102,Drama,True,"[es, en]"
97,Memento,2000,113,Thriller,True,"[es, en]"
98,Eternal Sunshine of the Spotless Mind,2004,108,Drama,False,"[es, en]"


In [45]:
## Convertir la columna subtitulos de formato lista a texto para que se pueda insertar en SQL

tabla_datos_pelis_act['subtitulos'] = tabla_datos_pelis_act['subtitulos'].apply(
    lambda x:
      ",".join(x)
      if isinstance(x, list)
        else x
)


In [42]:
## Crear tabla peliculas en la base de datos
mycursor.execute("USE bd_peliculas_adalab")

query = '''CREATE TABLE peliculas (
    id INT AUTO_INCREMENT PRIMARY KEY,
    titulo VARCHAR(255) NOT NULL,
    año INT,
    duracion INT,
    genero VARCHAR(100),
    contenido_para_adultos BOOLEAN,
    subtitulos VARCHAR(50)
);'''

mycursor.execute(query)
print("Tabla creada exitosamente")

Tabla creada exitosamente


3. INSERTAR LOS DATOS EN LA BASE DE DATOS CREADA: bd_peliculas_adalab

In [44]:
## Insertar datos
# Query de inserción
query_insertar = """
INSERT INTO peliculas (titulo,año,duracion,genero,contenido_para_adultos,subtitulos) 
VALUES (%s, %s, %s, %s, %s, %s)
"""

datos = tabla_datos_pelis_act.values.tolist()


# Ejecutar inserción
mycursor.executemany(query_insertar, datos)
cnx.commit()

print(f"{mycursor.rowcount} registros insertados")

print(f"{mycursor.rowcount} registros insertados")

100 registros insertados
100 registros insertados


FASE 4: OBTENER INFORMACION A PARTIR DE LOS DATOS
Una vez que tenemos toda la información, vamos a responder las siguientes preguntas utilizando consultas
en SQL

In [ ]:
##¿Cuántas películas tienen una duración superior a 120 minutos?
